## CitiBike data is collected for 1 year 2018
https://ride.citibikenyc.com/system-data

Data Schema:
Trip Duration (seconds)

*   Start Time and Date
*   Stop Time and Date
*   Start Station Name
*   End Station Name
*   Station ID
*   Station Lat/Long
*   Bike ID
*   User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual    Member)
*   Gender (Zero=unknown; 1=male; 2=female)
*   Year of Birth
 

In [1]:
import pandas as pd
import numpy as np
import haversine as hs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline
from glob import iglob

import warnings
warnings.filterwarnings("ignore")


In [2]:
#!pip install haversine

In [3]:
path = r'D:\Beuth Sems\Thesis\dataset\JC-2018*.csv'

all_rec = iglob(path, recursive=True)     
dataframes = (pd.read_csv(f) for f in all_rec)
tripdata = pd.concat(dataframes, ignore_index=True)

  

In [4]:
tripdata.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,3.538920e+05,353892.000000,353892.000000,353892.000000,353892.000000,353892.000000,353892.000000,353892.000000,353892.000000,353892.000000
mean,6.733930e+02,3264.996606,40.722724,-74.046039,3258.408418,40.722326,-74.045505,29452.498808,1980.387875,1.152388
std,7.004022e+03,138.429108,0.007251,0.010755,147.610023,0.007095,0.010759,2529.992371,10.268528,0.500198
min,6.100000e+01,3183.000000,40.692640,-74.096937,127.000000,40.679331,-74.096937,14697.000000,1887.000000,0.000000
25%,2.280000e+02,3192.000000,40.718211,-74.050444,3186.000000,40.717732,-74.049968,26315.000000,1974.000000,1.000000
50%,3.350000e+02,3205.000000,40.721525,-74.043845,3203.000000,40.721124,-74.043117,29493.000000,1983.000000,1.000000
75%,5.490000e+02,3272.000000,40.727224,-74.038051,3272.000000,40.727224,-74.037683,29679.000000,1988.000000,1.000000
max,2.061932e+06,3694.000000,40.748716,-74.032108,3694.000000,40.814326,-73.947821,35009.000000,2002.000000,2.000000


In [5]:
tripdata.isnull().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

In [6]:
tripdata.shape

(353892, 15)

In [7]:
tripdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353892 entries, 0 to 353891
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             353892 non-null  int64  
 1   starttime                353892 non-null  object 
 2   stoptime                 353892 non-null  object 
 3   start station id         353892 non-null  int64  
 4   start station name       353892 non-null  object 
 5   start station latitude   353892 non-null  float64
 6   start station longitude  353892 non-null  float64
 7   end station id           353892 non-null  int64  
 8   end station name         353892 non-null  object 
 9   end station latitude     353892 non-null  float64
 10  end station longitude    353892 non-null  float64
 11  bikeid                   353892 non-null  int64  
 12  usertype                 353892 non-null  object 
 13  birth year               353892 non-null  int64  
 14  gend

### calculate travelled distances based on lat and lon


In [8]:
#!pip install haversine

In [9]:
from haversine import Unit
import haversine as hs

def find_distance(slat, slon, elat, elon):
    loc1=(slat,slon)
    loc2=(elat, elon)
    return round(hs.haversine(loc1,loc2,unit=Unit.MILES),1)

In [10]:
## dinstace in miles
tripdata['dist'] = tripdata.apply(lambda r: find_distance(r['start station latitude'],r['start station longitude'],r['end station latitude'],r['end station longitude']) , axis=1)

In [11]:
tripdata.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,dist
0,932,2018-01-01 02:06:17.5410,2018-01-01 02:21:50.0270,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1,0.9
1,550,2018-01-01 12:06:18.0390,2018-01-01 12:15:28.4430,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31845,Subscriber,1969,2,0.9
2,510,2018-01-01 12:06:56.9780,2018-01-01 12:15:27.8100,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31708,Subscriber,1946,1,0.9
3,354,2018-01-01 14:53:10.1860,2018-01-01 14:59:05.0960,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,31697,Subscriber,1994,1,0.4
4,250,2018-01-01 17:34:30.1920,2018-01-01 17:38:40.9840,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,31861,Subscriber,1991,1,0.2


## Explode the start and end date into different columns

In [14]:
tripdata['starttime'] = pd.to_datetime(tripdata['starttime'])
tripdata['stoptime'] = pd.to_datetime(tripdata['stoptime'])
tripdata['date'] = tripdata['starttime'].dt.date
tripdata["month"] = tripdata["starttime"].dt.month_name()
tripdata['day'] = tripdata["starttime"].dt.day_name()
tripdata["hour"] = tripdata["starttime"].dt.hour
tripdata["min"] = tripdata["starttime"].dt.minute
tripdata["year"] =tripdata['starttime'].dt.year

In [15]:
tripdata.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'dist', 'birthyear', 'date', 'month', 'day',
       'hour', 'min', 'year'],
      dtype='object')

In [16]:
tripdata['birthyear'] = pd.to_numeric(tripdata['birth year'], downcast='integer')
tripdata['years_old'] = tripdata['year'] - tripdata['birthyear'] 
Age_Groups = ["<20", "20-29", "30-39", "40-49", "50-59", "60-64","65+"]
Age_Groups_Limits = [0, 20, 30, 40, 50, 60, 65, np.inf]
Age_Min = 0
Age_Max = 100
tripdata["age_group"] = pd.cut(tripdata["years_old"], Age_Groups_Limits, labels=Age_Groups)

In [17]:
# ### grouby the data to get the counts 
# transformed_tripdata = tripdata.groupby(['start station id',
#        'start station name', 'start station latitude',
#        'start station longitude', 'end station id', 'end station name',
#        'end station latitude', 'end station longitude', 'bikeid', 'usertype',
#        'birth year', 'gender', 'dist', 'date', 'month', 'day', 'hour', 'min','year'])['tripduration'].agg('sum').reset_index()

In [18]:
tripdata.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,dist,birthyear,date,month,day,hour,min,year,years_old,age_group
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,0.9,1992,2018-01-01,January,Monday,2,6,2018,26,20-29
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,0.9,1969,2018-01-01,January,Monday,12,6,2018,49,40-49
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,0.9,1946,2018-01-01,January,Monday,12,6,2018,72,65+
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,...,0.4,1994,2018-01-01,January,Monday,14,53,2018,24,20-29
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,...,0.2,1991,2018-01-01,January,Monday,17,34,2018,27,20-29


In [21]:
tripdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353892 entries, 0 to 353891
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   tripduration             353892 non-null  int64         
 1   starttime                353892 non-null  datetime64[ns]
 2   stoptime                 353892 non-null  datetime64[ns]
 3   start station id         353892 non-null  int64         
 4   start station name       353892 non-null  object        
 5   start station latitude   353892 non-null  float64       
 6   start station longitude  353892 non-null  float64       
 7   end station id           353892 non-null  int64         
 8   end station name         353892 non-null  object        
 9   end station latitude     353892 non-null  float64       
 10  end station longitude    353892 non-null  float64       
 11  bikeid                   353892 non-null  int64         
 12  usertype        

In [22]:
#!pip install holidays

In [23]:
from datetime import date
import holidays

# Select country
us_holidays = holidays.US()
data=[]
for ptr in holidays.US(years = 2018).items():
    data.append([ptr[0],ptr[1]])
    
df_holiday = pd.DataFrame(data, columns=['date','holiday'])
  
# print dataframe.
df_holiday   

,date,holiday
0,2018-01-01,New Year's Day
1,2018-01-15,Martin Luther King Jr. Day
2,2018-02-19,Washington's Birthday
3,2018-05-28,Memorial Day
4,2018-07-04,Independence Day
5,2018-09-03,Labor Day
6,2018-10-08,Columbus Day
7,2018-11-11,Veterans Day
8,2018-11-12,Veterans Day (Observed)
9,2018-11-22,Thanksgiving


### Add holiday details to trip data

In [24]:
df = pd.merge(
    left=tripdata, 
    right=df_holiday, 
    on='date',
    how='left'
)

In [25]:
df['holiday'].fillna('No Holiday', inplace=True)

In [26]:
df.head(3)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,birthyear,date,month,day,hour,min,year,years_old,age_group,holiday
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,1992,2018-01-01,January,Monday,2,6,2018,26,20-29,New Year's Day
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,1969,2018-01-01,January,Monday,12,6,2018,49,40-49,New Year's Day
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,1946,2018-01-01,January,Monday,12,6,2018,72,65+,New Year's Day


In [27]:
df['date'] = pd.to_datetime(df['date'])

## Read a weather data

In [28]:
weather_data = pd.read_csv("../dataset/nyc_weather_data.csv")
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetime        365 non-null    object 
 1   tempmax         365 non-null    float64
 2   tempmin         365 non-null    float64
 3   temp            365 non-null    float64
 4   feelslike       365 non-null    float64
 5   precip          365 non-null    float64
 6   dew             365 non-null    float64
 7   humidity        365 non-null    float64
 8   snow            365 non-null    float64
 9   snowdepth       365 non-null    float64
 10  windspeed       365 non-null    float64
 11  visibility      365 non-null    float64
 12  solarradiation  365 non-null    float64
 13  cloudcover      365 non-null    float64
 14  conditions      365 non-null    object 
 15  description     365 non-null    object 
dtypes: float64(13), object(3)
memory usage: 45.8+ KB


In [29]:
weather_data.isnull().sum()

datetime          0
tempmax           0
tempmin           0
temp              0
feelslike         0
precip            0
dew               0
humidity          0
snow              0
snowdepth         0
windspeed         0
visibility        0
solarradiation    0
cloudcover        0
conditions        0
description       0
dtype: int64

In [30]:
weather_data['date'] = pd.to_datetime(weather_data['datetime'])
weather_data.dtypes

datetime                  object
tempmax                  float64
tempmin                  float64
temp                     float64
feelslike                float64
precip                   float64
dew                      float64
humidity                 float64
snow                     float64
snowdepth                float64
windspeed                float64
visibility               float64
solarradiation           float64
cloudcover               float64
conditions                object
description               object
date              datetime64[ns]
dtype: object

In [31]:
complete_df = pd.merge(
    left=df, 
    right=weather_data, 
    on='date',
    how='left'
)

In [32]:
complete_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,dew,humidity,snow,snowdepth,windspeed,visibility,solarradiation,cloudcover,conditions,description
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,-19.9,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,-19.9,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,-19.9,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,...,-19.9,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,...,-19.9,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.


In [33]:
complete_df.isnull().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dist                       0
birthyear                  0
date                       0
month                      0
day                        0
hour                       0
min                        0
year                       0
years_old                  0
age_group                  0
holiday                    0
datetime                   0
tempmax                    0
tempmin                    0
temp                       0
feelslike                  0
precip                     0
dew                        0
humidity                   0
snow          

## add seasonality data
1. spring (March-May)
2. summer (June-August)
3. autumn (September-November) 
4. winter (December-February

In [34]:
def add_seasonality(month):
    if (month == 'March' or month == 'April' or month == 'May'):
        return 'spring'
    elif month == 'June' or month == 'July' or month == 'August':
        return 'summer'
    elif month == 'September' or month == 'October' or month == 'November':
        return 'autumn'
    elif month == 'December' or month == 'January' or month == 'February':
        return 'winter'

In [35]:

complete_df['seasons'] = complete_df.apply(lambda r: add_seasonality(r['month']) , axis=1)

In [36]:
complete_df.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,humidity,snow,snowdepth,windspeed,visibility,solarradiation,cloudcover,conditions,description,seasons
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.,winter
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.,winter
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,...,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.,winter
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,...,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.,winter
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,...,47.8,0.0,0.1,18.5,16.0,106.7,0.3,Clear,Clear conditions throughout the day.,winter


In [37]:
complete_df['seasons'].unique()

array(['winter', 'spring', 'summer', 'autumn'], dtype=object)

In [38]:
complete_df.to_csv("../dataset/complete_bike_sharing.csv", index=False,)